## Retreiving and cleaning data

In [2]:
import pandas as pd
from os import path

CENSUS_KEY = "5c32f4eee940b3dee7a2d96f920828e469ad9255"

### Combining three 5-year ACS dataset into one

**Source:** [Census Flow API](https://api.census.gov/data/2020/acs/flows)

**Raw data:**
  - acs_desoto_flows_20XX.csv - 5-year American Community Survey: Migration Flows for Desoto County, Miss.
  
**Processed data:**
  - acs_desoto_flows_15y.csv - 15-year migration flows, county to DeSoto County, Miss.

In [44]:
survey_years = ["2010", "2015", "2020"]
survey_dfs = []

for year in survey_years:
    # checking if data is already downloaded
    if not path.isfile(f"../data/raw/acs_desoto_flows_{year}.csv"):
        # using census flow api
        url = f"https://api.census.gov/data/{year}/acs/flows?get=FULL2_NAME,GEOID2,MOVEDIN,MOVEDOUT,MOVEDNET&for=county:033&in=state:28&SUMLEV2=50&key={CENSUS_KEY}"
        df = pd.read_json(url)
        df.to_csv(f"../data/raw/acs_desoto_flows_{year}.csv",  index=False)
    else:
        df = pd.read_csv(f"../data/raw/acs_desoto_flows_{year}.csv")
    df.columns = df.iloc[0, :]  # making first row headers
    df.drop(0, axis=0, inplace=True)  # deleting first row
    survey_dfs.append(df)

acs_desoto_flows_15y = pd.concat(survey_dfs) # combine calls
acs_desoto_flows_15y.head(2)

,FULL2_NAME,GEOID2,MOVEDIN,MOVEDOUT,MOVEDNET,SUMLEV2,state,county
1,"Franklin County, Alabama",01059,11,0,11,50,28,033
2,"Jefferson County, Alabama",01073,0,24,-24,50,28,033


In [45]:
# remove unnecessary columns
acs_desoto_flows_15y = acs_desoto_flows_15y[
    ["GEOID2", "FULL2_NAME", "MOVEDIN", "MOVEDOUT", "MOVEDNET"]
]  

# rename columns
acs_desoto_flows_15y.columns = [
    "id",
    "county",
    "movedin",
    "movedout",
    "movednet",
] 

# correct typo in data
acs_desoto_flows_15y.loc[acs_desoto_flows_15y['id'] == '29510', 'county'] = "St. Louis City, Missouri"

Per [Census documentation](https://www.census.gov/data/developers/data-sets/acs-migration-flows.2020.html) numbers are annual, so multiplying the numbers by five would estimate total number of movers over the period.

> The flow estimates resemble the annual number of movers between counties for the 5-year period data was collected.

In [46]:
# multiples each survey estimate by 5 and sums all
acs_desoto_flows_15y = acs_desoto_flows_15y.groupby(["id", "county"]).agg(
    lambda x: 5 * sum(x.apply(pd.to_numeric))
)  

In [47]:
# save file
acs_desoto_flows_15y.to_csv("../data/processed/acs_desoto_flows_15y.csv")
acs_desoto_flows_15y.head(5)

,,movedin,movedout,movednet
id,county,,,
01003,"Baldwin County, Alabama",0,275,-275
01015,"Calhoun County, Alabama",0,35,-35
01033,"Colbert County, Alabama",0,90,-90
01039,"Covington County, Alabama",25,0,25
01045,"Dale County, Alabama",0,10,-10


### Every available survey 5-year estimate for Shelby County to DeSoto Flows

**Source:** [Census Flow API](https://api.census.gov/data/2020/acs/flows)

**Raw data**
  - acs_desoto_flows_net_20XX.csv - 5-year American Community Survey: Migration Flows for Desoto County, Miss. only net migration

**Processed data**
- acs_desoto_flows_net_shelby_rolling_avg.csv - 5-year rolling average of net migration for DeSoto County, Miss. and Shelby County, Tenn. vs all other counties

In [3]:
survey_dfs = []

for year in range(2010,2021):
    if not path.isfile(f"../data/raw/acs_desoto_flows_net_{year}.csv"):
        url = f"https://api.census.gov/data/{year}/acs/flows?get=GEOID2,MOVEDNET&for=county:033&in=state:28&SUMLEV2=50&key={CENSUS_KEY}"
        df = pd.read_json(url)
        df.to_csv(f"../data/raw/acs_desoto_flows_net_{year}.csv",  index=False)
    else:
        df = pd.read_csv(f"../data/raw/acs_desoto_flows_net_{year}.csv")
    df.columns = df.iloc[0, :]  # making first row headers
    df.drop(0, axis=0, inplace=True)  # deleting first row
    df['year'] = year # adding year column
    survey_dfs.append(df) 

acs_desoto_flows_net_shelby_rolling_avg = pd.concat(survey_dfs) # combine calls
acs_desoto_flows_net_shelby_rolling_avg.head(2)

,GEOID2,MOVEDNET,SUMLEV2,state,county,year
1,01059,11,50,28,033,2010
2,01073,-24,50,28,033,2010


In [4]:
# remove unnecessary columns
acs_desoto_flows_net_shelby_rolling_avg = acs_desoto_flows_net_shelby_rolling_avg[
    ["GEOID2", "year", "MOVEDNET"]
]  

# rename columns
acs_desoto_flows_net_shelby_rolling_avg.columns = [
    "id",
    "year",
    "movednet",
]  

# movednet to numeric
acs_desoto_flows_net_shelby_rolling_avg["movednet"] = (
    acs_desoto_flows_net_shelby_rolling_avg["movednet"].apply(pd.to_numeric)
)

In [5]:
# create column only shelby numbers
acs_desoto_flows_net_shelby_rolling_avg["shelby"] = (
    acs_desoto_flows_net_shelby_rolling_avg.apply(
        lambda x: x["movednet"] if x["id"] == "47157" else 0, axis=1
    )
) 

# create column only non-shelby numbers
acs_desoto_flows_net_shelby_rolling_avg["no_shelby"] = (
    acs_desoto_flows_net_shelby_rolling_avg.apply(
        lambda x: x["movednet"] if x["id"] != "47157" else 0, axis=1
    )
)

# group by year to combine all non-shelby counties
acs_desoto_flows_net_shelby_rolling_avg = (
    acs_desoto_flows_net_shelby_rolling_avg.drop(["id", "movednet"], axis=1)
    .groupby("year")
    .sum()
    .reset_index()
)

In [6]:
# save file
acs_desoto_flows_net_shelby_rolling_avg.to_csv('../data/processed/acs_desoto_flows_net_shelby_rolling_avg.csv', index=False)
acs_desoto_flows_net_shelby_rolling_avg.head(5)

,year,shelby,no_shelby
0,2010,2145,611
1,2011,1501,140
2,2012,1499,-897
3,2013,1135,812
4,2014,624,409


### Race of migrants county-to-county flows

**Source:** [Census Flow API](https://api.census.gov/data/2020/acs/flows)

Per [Census documentation](https://www.census.gov/data/developers/data-sets/acs-migration-flows.2020.html) race of migrants was collected for two surveys, 2006-2010 ACS and 2011-2015 ACS:


> Characteristics are included for the following years:
> - 2011-2015 ACS: Age, Sex, Race, Hispanic origin
> - 2010-2014 ACS: Relationship to householder, Household type, Housing tenure
> - 2009-2013 ACS: Ability to speak English, Place of birth, Year in the United States (or Puerto Rico)
> - 2008-2012 ACS: Employment status, Occupation, Work status
> - 2007-2011 ACS: Educational Attainment, Personal income, Household income
> - 2006-2010 ACS: Age, Sex, Race, Hispanic origin

**Raw data:**
  - acs_desoto_shelby_movednet_20XX_chars.csv - 5-year American Community Survey: Migration Flows for Desoto County, Miss. and Shelby County, Tenn., including age, sex, race, Hispanic origin characteristics
  
**Processed data:**
  - acs_desoto_shelby_movednet_10y_chars.csv - 2006-2015 migration flows, Shelby County, Tenn. to DeSoto County, Miss., including age, sex, race, Hispanic origin characteristics

In [32]:
survey_years = ["2010", "2015"]
survey_dfs = []

for year in survey_years:
    # checking if data is already downloaded
    if not path.isfile(f"../data/raw/acs_desoto_shelby_movednet_{year}_chars.csv"):
        # using census flow api
        url = f"https://api.census.gov/data/{year}/acs/flows?get=FULL2_NAME,GEOID2,MOVEDNET,RACE,AGE,SEX,{'HSGP' if  year == '2015' else 'HISP_ORIGIN'}&for=county:033&in=state:28&SUMLEV2=50&GEOID2=47157&key={CENSUS_KEY}"
        df = pd.read_json(url)
        df.to_csv(f"../data/raw/acs_desoto_shelby_movednet_{year}_chars.csv", index=False)
    else:
        df = pd.read_csv(f"../data/raw/acs_desoto_shelby_movednet_{year}_chars.csv")
    df.columns = ['FULL2_NAME', 'GEOID2', 'MOVEDNET', 'RACE', 'AGE', 'SEX', 'HISP_ORIGIN', 'SUMLEV2', 'GEOID2', 'state', 'county'] # making first row headers
    df.drop(0, axis=0, inplace=True)  # deleting first row
    df['year'] = year
    survey_dfs.append(df)

acs_desoto_shelby_movednet_10y_chars = pd.concat(survey_dfs) # combine calls
acs_desoto_shelby_movednet_10y_chars.head(2)

,FULL2_NAME,GEOID2,MOVEDNET,RACE,AGE,SEX,HISP_ORIGIN,SUMLEV2,GEOID2,state,county,year
1,"Shelby County, Tennessee",47157,2145,0,0,00,0,50,47157,28,033,2010
2,"Shelby County, Tennessee",47157,1150,0,0,01,0,50,47157,28,033,2010


In [34]:
# create characteristic code dictionary
char_codes = {
    "AGE": {
        "00" : "All",
        "01" : "1 to 4 years",
        "02": "5 to 17 years",
        "03": "18 to 19 years",
        "04": "20 to 24 years",
        "05": "25 to 29 years",
        "06": "30 to 34 years",
        "07": "35 to 39 years",
        "08": "40 to 44 years",
        "09": "45 to 49 years",
        "10": "50 to 54 years",
        "11": "55 to 59 years",
        "12": "60 to 64 years",
        "13": "65 to 69 years",
        "14": "70 to 74 years",
        "15": "75 years and over"
    },
    "SEX": {
        "00" : "All",
        "01": "Male",
        "02": "Female"
    },
    "RACE": {
        "00" : "All",
        "01": "White alone",
        "02": "Black or African American alone",
        "03": "Asian alone",
        "04": "Other race alone or Two or more races"
    },
    "HISP_ORIGIN": {
        "00" : "All",
        "01": "White alone, not Hispanic or Latino",
        "02": "Not white alone, not Hispanic or Latino",
        "03": "Hispanic or Latino"
    }
}

In [35]:
### switch out char codes for values
for k, d in char_codes.items():
    acs_desoto_shelby_movednet_10y_chars[k] = acs_desoto_shelby_movednet_10y_chars[k].apply(lambda x: d[str(x).zfill(2)] if d[str(x).zfill(2)] else pd.NA)

In [36]:
# 'all' chars are summary values, so removed, pivoted years to columns
acs_desoto_shelby_movednet_10y_chars = (
    acs_desoto_shelby_movednet_10y_chars.melt(
        value_vars=["RACE", "AGE", "SEX", "HISP_ORIGIN"], id_vars=["MOVEDNET", "year"]
    )
    .query('value != "All"')
    .pivot(values="MOVEDNET", index="value", columns="year")
    .apply(pd.to_numeric)
    .sort_values("2015", ascending=False)
    .reset_index()
)

# fix columns
acs_desoto_shelby_movednet_10y_chars[['value', '2010', '2015']]
acs_desoto_shelby_movednet_10y_chars.columns = ['chars', '2010', '2015']

,MOVEDNET,year,variable,value
0,2145,2010,RACE,All
1,1150,2010,RACE,All
2,995,2010,RACE,All
3,796,2010,RACE,All
4,1170,2010,RACE,All
...,...,...,...,...
195,156,2015,HISP_ORIGIN,All
196,-188,2015,HISP_ORIGIN,All
197,-157,2015,HISP_ORIGIN,"White alone, not Hispanic or Latino"
198,608,2015,HISP_ORIGIN,"Not white alone, not Hispanic or Latino"


In [14]:
acs_desoto_shelby_movednet_10y_chars.to_csv('../data/processed/acs_desoto_shelby_movednet_10y_chars.csv', index=False)
acs_desoto_shelby_movednet_10y_chars.head(5)

,chars,2010,2015
0,"Not white alone, not Hispanic or Latino",1170,608
1,Black or African American alone,953,546
2,25 to 29 years,577,171
3,Female,995,163
4,Asian alone,46,156


### County and state population estimates, 2000-2023

Note: more recent population data is not available through the api, so I'm using Census's FTP server to retrieve full datasets.

**Source**: [Census Population Estimates FTP server](https://www2.census.gov/programs-surveys/popest/datasets) - [Docs](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2023/CO-EST2023-ALLDATA.pdf)

**Raw data**
- co-est20XX-alldata.csv - Annual Resident Population Estimates for Counties

**Processed data**
- counties_pop_estimates_2000_2023

In [15]:
direct_urls = ["https://www2.census.gov/programs-surveys/popest/datasets/2020-2023/counties/totals/co-est2023-alldata.csv", "https://www2.census.gov/programs-surveys/popest/datasets/2010/2010-eval-estimates/co-est2010-alldata.csv", "https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/counties/totals/co-est2020-alldata.csv"]
dfs = []

for url in direct_urls:
    if not path.isfile(f"../data/raw/{url.split('/')[-1]}"):
        df = pd.read_csv(url, encoding='latin-1')
        df.to_csv(f"../data/raw/{url.split('/')[-1]}", index=False)
    else:
        df = pd.read_csv(f"../data/raw/{url.split('/')[-1]}")
    df = df.set_index(list(df.columns)[:7]).stack().reset_index()
    df.columns = ['sum_level', 'region', 'division', 'state', 'county', 'state_name', 'county_name', 'var', 'value']
    df.loc[:,'year'] = df['var'].str[-4:]
    df.loc[:,'var'] = df['var'].str[:-4]
    df = df[['sum_level', 'region', 'division','state', 'county', 'state_name', 'county_name', 'year', 'var', 'value']]
    dfs.append(df)

counties_pop_estimates_2000_2023 = pd.concat(dfs) # combine datasets
counties_pop_estimates_2000_2023.head(2)

,sum_level,region,division,state,county,state_name,county_name,year,var,value
0,40,3,6,1,0,Alabama,Alabama,2020,ESTIMATESBASE,5024294.0
1,40,3,6,1,0,Alabama,Alabama,2020,POPESTIMATE,5031864.0


In [26]:
# convert value, year to numeric
counties_pop_estimates_2000_2023['value'] = pd.to_numeric(counties_pop_estimates_2000_2023['value'], errors='coerce')
counties_pop_estimates_2000_2023['year'] = pd.to_numeric(counties_pop_estimates_2000_2023['year'], errors='coerce', downcast='integer')

In [27]:
counties_pop_estimates_2000_2023.to_csv('../data/processed/counties_pop_estimates_2000_2023.csv', index=False)
counties_pop_estimates_2000_2023.head(5)

,sum_level,region,division,state,county,state_name,county_name,year,var,value
0,40,3,6,1,0,Alabama,Alabama,2020,ESTIMATESBASE,5024294.0
1,40,3,6,1,0,Alabama,Alabama,2020,POPESTIMATE,5031864.0
2,40,3,6,1,0,Alabama,Alabama,2021,POPESTIMATE,5050380.0
3,40,3,6,1,0,Alabama,Alabama,2022,POPESTIMATE,5073903.0
4,40,3,6,1,0,Alabama,Alabama,2023,POPESTIMATE,5108468.0


### DeSoto County, Miss., Shelby County, Tenn. and Mississippi population estimates 1970-2023

Note: To retrieve early data, I retrieved data from FRED rather than directly from the Census because older population is less digestable and would take more cleaning or concateanting multiple api calls than more recent data.

**Source**: [Census Population Estimates, retrieved from FRED](https://fred.stlouisfed.org/series/MSDEPOP)

**Raw data**
- pop_est_desoto_shelby_ms_1970_2023.csv - Annual Resident Population Estimates for DeSoto County, Miss., Shelby County, Tenn. and Mississippi

**Processed data**
- pop_est_desoto_shelby_ms_1970_2023.csv - Annual Resident Population Estimates for DeSoto County, Miss., Shelby County, Tenn. and Mississippi

In [11]:
pop_est_desoto_shelby_ms_1970_2023 = pd.read_csv('../data/raw/pop_est_desoto_shelby_ms_1970_2023.csv')

# rename columns
pop_est_desoto_shelby_ms_1970_2023.columns = ['Date','Shelby County', 'DeSoto County', 'Mississippi']

# date to datatime value
pop_est_desoto_shelby_ms_1970_2023['Date'] = pd.to_datetime(pop_est_desoto_shelby_ms_1970_2023['Date'])

In [12]:
pop_est_desoto_shelby_ms_1970_2023.to_csv('../data/processed/pop_est_desoto_shelby_ms_1970_2023.csv', index= False)
pop_est_desoto_shelby_ms_1970_2023.head(5)

,Date,Shelby County,DeSoto County,Mississippi
0,1970-01-01,722.111,35.885,2216.994
1,1971-01-01,733.500,36.800,2265.432
2,1972-01-01,741.100,39.900,2307.117
3,1973-01-01,734.600,45.200,2349.546
4,1974-01-01,742.100,47.500,2378.268


### Race population data

Note: I retrieved this data from FRED rather directly from the Census because it was easier. Not a best practice.

**Source:** [U.S. Census Bureau, 5-year County Population Estimates By Race And Ethnicity, retrieved from FRED](https://fred.stlouisfed.org/series/B03002004E047157)

**Raw data**
- desoto_race_pop_percent_change.csv - Percent change in population of Black alone and White alone residents in DeSoto County, Miss.
- shelby_race_pop_percent_change.csv - Percent change in population of Black alone and White alone residents in Shelby County, Miss.

**Processed data**
- desoto_race_pop_percent_change.csv - Percent change in population of Black alone and White alone residents in DeSoto County, Miss.
- shelby_race_pop_percent_change.csv - Percent change in population of Black alone and White alone residents in Shelby County, Miss.

In [7]:
desoto_race_pop = pd.read_csv('../data/raw/desoto_race_pop_percent_change.csv')
shelby_race_pop = pd.read_csv('../data/raw/shelby_race_pop_percent_change.csv')

desoto_race_pop.columns = ['date', 'white', 'black']
shelby_race_pop.columns = ['date', 'black', 'white']
desoto_race_pop = desoto_race_pop[['date', 'black', 'white']]

shelby_race_pop['date'] = pd.to_datetime(shelby_race_pop['date'])
desoto_race_pop['date'] = pd.to_datetime(desoto_race_pop['date'])

shelby_race_pop.to_csv('../data/processed/shelby_race_pop_percent_change.csv', index=False)
desoto_race_pop.to_csv('../data/processed/desoto_race_pop_percent_change.csv', index=False)

display(shelby_race_pop.head(5))
display(desoto_race_pop.head(5))

,date,black,white
0,2010-01-01,2.15956,-3.34521
1,2011-01-01,0.64943,-0.89416
2,2012-01-01,0.82953,-0.99581
3,2013-01-01,1.08500,-0.79204
4,2014-01-01,0.78690,-0.77903


,date,black,white
0,2010-01-01,1.94165,2.24176
1,2011-01-01,5.48313,0.87043
2,2012-01-01,4.74106,0.45846
3,2013-01-01,5.22095,0.51099
4,2014-01-01,11.56216,0.31205
